In [43]:
import os
import pandas as pd
import random
from langid.langid import LanguageIdentifier, model
from tqdm import tqdm
import re

In [44]:

identifier = LanguageIdentifier.from_modelstring(model, norm_probs=True)


input_folder = "../dataset/twitter100m_tweets/data"
output_folder = "../dataset/twitter100m_tweets_filtered/data"
os.makedirs(output_folder, exist_ok=True)

In [45]:

def preprocess_tweet(tweet):
    
   
    tweet = re.sub(r"http\S+|@\S+|#\S+", "", tweet)  
    tweet = re.sub(r"[^\w\s]", "", tweet) 
    return tweet.strip()


def detect_language(tweet):
    try:
        if isinstance(tweet, str):

            cleaned_tweet = tweet.strip()
            if re.fullmatch(r"http\S+", cleaned_tweet):  
                return "unknown"  
            
            tweet = preprocess_tweet(tweet)  
            lang, _ = identifier.classify(tweet)
            return lang
    except:
        return "unknown"


def is_non_english_or_spam_user(user_tweets, sample_size=20, threshold=17, spam_threshold=17):
    """
    Determine if a user is predominantly non-English or spamming links.

    Parameters:
        user_tweets (list): List of tweets by the user.
        sample_size (int): Number of tweets to sample for analysis.
        threshold (int): Number of non-English tweets needed to classify as non-English.
        spam_threshold (int): Number of "link" tweets needed to classify as spam.

    Returns:
        bool: True if the user is predominantly non-English or spamming, False otherwise.
    """
    
    sampled_tweets = random.sample(list(user_tweets), min(len(user_tweets), sample_size))
    detected_languages = [detect_language(tweet) for tweet in sampled_tweets]

   
    non_english_count = sum(1 for lang in detected_languages if lang not in {"en", "link"})
    
   
    spam_count = sum(1 for lang in detected_languages if lang == "link")

   
    return non_english_count >= threshold or spam_count >= spam_threshold



def is_filtered(file):
    filtered_file = os.path.join(output_folder, f"filtered_{file}")
    return os.path.exists(filtered_file)

In [ ]:

for file in tqdm(os.listdir(input_folder), desc="Checking Files"):
    if file.endswith(".parquet") and not is_filtered(file):
        file_path = os.path.join(input_folder, file)
        print(f"Processing {file_path}...")

        
        df = pd.read_parquet(file_path)

        
        tqdm.pandas(desc="Filtering Users")
        user_groups = df.groupby("user")  
        non_english_users = user_groups["tweet"].progress_apply(is_non_english_or_spam_user)
        non_english_users = non_english_users[non_english_users].index  
        
        
        english_user_tweets = df[~df["user"].isin(non_english_users)]
        
        
        output_file = os.path.join(output_folder, f"filtered_{file}")
        english_user_tweets.to_parquet(output_file, index=False)
        print(f"Saved filtered file: {output_file}")

        

Checking Files:   0%|          | 0/41 [00:00<?, ?it/s]

Processing ../dataset/twitter100m_tweets/data\train-00001-of-00041-4491d6daeecba187.parquet...


Checking Files:   5%|▍         | 2/41 [02:48<54:48, 84.32s/it]

Saved filtered file: ../dataset/twitter100m_tweets_filtered/data\filtered_train-00001-of-00041-4491d6daeecba187.parquet
Processing ../dataset/twitter100m_tweets/data\train-00002-of-00041-6f8b4e61b330413a.parquet...


Checking Files:   7%|▋         | 3/41 [05:16<1:10:16, 110.97s/it]

...

Saved filtered file: ../dataset/twitter100m_tweets_filtered/data\filtered_train-00038-of-00041-08d9366060cda96a.parquet
Processing ../dataset/twitter100m_tweets/data\train-00039-of-00041-d3dbd276c36d5b35.parquet...


Checking Files:  98%|█████████▊| 40/41 [1:31:02<02:21, 141.85s/it]

Saved filtered file: ../dataset/twitter100m_tweets_filtered/data\filtered_train-00039-of-00041-d3dbd276c36d5b35.parquet
Processing ../dataset/twitter100m_tweets/data\train-00040-of-00041-9a723429a2a70e30.parquet...


Checking Files: 100%|██████████| 41/41 [1:33:27<00:00, 136.76s/it]

Saved filtered file: ../dataset/twitter100m_tweets_filtered/data\filtered_train-00040-of-00041-9a723429a2a70e30.parquet
